In [1]:
%gui qt

import nibabel as nib
import napari
import numpy as np
import os

viewer = napari.Viewer()

raw_img_path = "/Users/dbattagodage/Desktop/Datasets/nnUnet_raw/Dataset352_IF/imagesTr" 
gen_seg_path = None#"/home/cellsmb/Desktop/Dinuka/Image_Analysis/Model_results/Model_results/IF_352_6img_results/all_segs_bg"
raw_seg_path = "/Users/dbattagodage/Desktop/Datasets/nnUnet_raw/Dataset352_IF/labelsTr"

files =[]
gen_map_labels_if_available = []
raw_map_labels_if_available = []

# primary = np.array([[1,0,0],[0,1/0.27,0],[0,0,1]])
# primary = np.array([[-0.27,0,0],[0,0.27,0],[0,0,-11]])
primary = None

if raw_seg_path is not None:
    assert os.path.exists(raw_img_path), f"Raw image path {raw_img_path} does not exist."
    files = os.listdir(raw_seg_path)
elif gen_seg_path is not None:
    files = os.listdir(gen_seg_path)
else:
    files = os.listdir(raw_img_path)
files.sort()
files = ["IF_Tr_0006.nii.gz"]

def napari_visualisation(IM_ID: int, ID_from_segs: bool = True, iteration: int = 0,show_seg: bool = True):
    global gen_map_labels_if_available, raw_map_labels_if_available

    nii_path_img1 = f"{raw_img_path}/IF_Tr_{IM_ID:04d}_0000.nii.gz"
    nii_path_img2 = f"{raw_img_path}/IF_Tr_{IM_ID:04d}_0001.nii.gz"
    img1 = nib.load(nii_path_img1).get_fdata().astype(np.uint8)
    img2 = nib.load(nii_path_img2).get_fdata().astype(np.uint8) 
    viewer.add_image(img1, name='Endocardium_%d'%IM_ID, colormap='gray', blending='additive',affine=primary)
    viewer.add_image(img2, name='Myocardium_%d'%IM_ID, colormap='gray', blending='additive',affine=primary)

    if ID_from_segs and show_seg:
        if gen_seg_path is not None:
            seg_path_gen = f"{gen_seg_path}/IF_Tr_{IM_ID:04d}.nii.gz"
            gen_segm = nib.load(seg_path_gen).get_fdata().astype(np.uint8)
            if iteration == 0:
                gen_map_labels_if_available= [0,1,2,3,4]
            add_labels_to_viewer(gen_segm, "gen_seg", gen_map_labels_if_available)

        if raw_seg_path is not None:
            seg_path_raw = f"{raw_seg_path}/IF_Tr_{IM_ID:04d}.nii.gz"
            raw_segm = nib.load(seg_path_raw).get_fdata().astype(np.uint8)
            if iteration == 0:
                raw_map_labels_if_available= [0,1,2,3,4]
            add_labels_to_viewer(raw_segm, "raw_seg", raw_map_labels_if_available)
    
def add_labels_to_viewer(segmentation, name, labels):
    for r in range(1,2):
        label_of_interest = labels[r]
        mask1 = (segmentation== label_of_interest).astype(np.uint8) * label_of_interest
        viewer.add_labels(mask1, name=f"{name}_{label_of_interest}_{IM_ID}", blending="translucent", opacity=0.7, affine=primary)

iteration = 0
for IMAGE_FILE in files:
    if IMAGE_FILE.endswith('.nii.gz'):
        split_name = IMAGE_FILE.split('_')
        if len(split_name) == 3:
            IM_ID = int(split_name[2].split('.')[0])
            napari_visualisation(IM_ID,True, iteration)
            iteration += 1
        elif len(split_name) == 4:
            if iteration%3==0:
                IM_ID = int(split_name[2])
                napari_visualisation(IM_ID, False)
            iteration += 1